In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import datetime as dt
import logging
import warnings
warnings.filterwarnings('ignore')
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import catboost
from catboost import CatBoostRegressor
from catboost import Pool, cv
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [ ]:
### Read Data 
items=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
item_cat=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
train_df=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
shops=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
test_df=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
submission=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')

In [ ]:
display(items.shape)
display(item_cat.shape)
display(train_df.shape)
display(shops.shape)
display(test_df.shape)
display(submission.shape)

In [ ]:
print(items.head())
print(item_cat.head())
print(train_df.head())
print(shops.head())
print(test_df.head())
print(submission.head())

In [ ]:
train_df.describe()

In [ ]:
train_df.isnull().sum()## No nulls present

In [ ]:
train_df.head()
## dropping the columns not required
train_df.drop(['date_block_num','item_price'],axis=1,inplace=True)

In [ ]:
### Convert the date column to pandas datetime type
train_df['date'] = pd.to_datetime(train_df['date'])

In [ ]:
train_df.head()
## Get month,year only
train_df['month']=train_df['date'].dt.month
train_df['year']=train_df['date'].dt.year

In [ ]:
train_df

In [ ]:
# train_df.drop('date',axis=1,inplace=True)
train_df['date'] = train_df['date'].apply(lambda x: x.strftime('%Y-%m'))

In [ ]:
train_df.head()

In [ ]:
### Aggregate the data by date,shopid and item id
data = train_df.groupby(['date','shop_id','item_id']).sum()

In [ ]:
data = data.pivot_table(index=['shop_id','item_id'], columns='date', values='item_cnt_day', fill_value=0)

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data

In [ ]:
display(test_df.shape)
test_df=pd.merge(test_df, data, on=['shop_id','item_id'], how='left')
display(test_df.shape)
test_df = test_df.fillna(0)

In [ ]:
test_df.T

In [ ]:
data.T

In [ ]:
# split into train and test sets
Y_train = data['2015-12'].values
X_train = data.drop(['2015-12'], axis = 1)
X_test = test_df
print(X_train.shape, Y_train.shape)
print(X_test.shape)

### Split the data into train and test set from train_df

In [ ]:
x_train, x_test, y_train, y_test = train_test_split( X_train, Y_train, test_size=0.20, random_state=123)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

### Create a benchmark Catboost regressor model

In [ ]:
## Prepara a pool of train and validation set
pool_train=Pool(x_train,y_train)
pool_val=Pool(x_test,y_test)

In [ ]:
### Define a cv function to fit on data and find the optimal number of iteration keeping other parameters fixed
### Function takes input = catboost object with default params , train data ,train y data 
def modelfit(params,poolX,useTrainCV=True,cv_folds=5,early_stopping_rounds=40):
    if useTrainCV:
        cvresult = cv(params=params, pool=poolX,nfold=cv_folds,early_stopping_rounds=early_stopping_rounds,plot=True)
    return cvresult ## return dataframe for the iteration till the optimal iteration is reached

In [ ]:
## Prepara a cv class
params={
    'loss_function':'RMSE'
}

In [ ]:
### Object return the optimal number of trees to grow
n_est=modelfit(params,pool_train)

In [ ]:
n_est.shape[0]### best iteration =664

### Fit the model with 600 iteration

In [ ]:
### Fit the model with iteration=664
cboost1=CatBoostRegressor(iterations=664,loss_function='RMSE',random_seed=123)
cboost1.fit(x_train,y_train)

In [ ]:
#Predict training set:
train_predictions = cboost1.predict(x_train)
#Print model report:
print("\nModel Report Train")
print("Root Mean Square Error : %.4g" % metrics.mean_squared_error(y_train, train_predictions))
print("R^2 Score (Train): %f" % metrics.r2_score(y_train, train_predictions))

#### On Test set

In [ ]:
#Predict training set:
test_predictions = cboost1.predict(x_test)
#Print model report:
print("\nModel Report Test")
print("Root Mean Square Error : %.4g" % metrics.mean_squared_error(y_test, test_predictions))
print("R^2 Score (Train): %f" % metrics.r2_score(y_test, test_predictions))

### Prediction on X_test

In [ ]:
X_train

In [ ]:
prediction = cboost1.predict(X_test)

In [ ]:
prediction = list(map(round, prediction))

In [ ]:
len(prediction)

### Submission Data

In [ ]:
submission.shape

In [ ]:
submission
submission['item_cnt_month'] = prediction
submission.to_csv('prediction.csv', index=False)

In [ ]:
submission.tail()

In [ ]:
submission.head(50)